In [2]:
import pandas as pd
import sqlite3
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity



# Read data
dfm = pd.read_csv('/workspaces/machine-learning-python-template-ds-2023/Ryan/raw/movies.csv')
dfc = pd.read_csv('/workspaces/machine-learning-python-template-ds-2023/Ryan/raw/credits.csv')

# See info
print(dfm.info())
print(dfc.info())
# Create a SQLite database connection
conn = sqlite3.connect(':memory:')

# Convert the dataframes to SQL tables
dfm.to_sql('movies', conn, index=False, if_exists='replace')
dfc.to_sql('credits', conn, index=False, if_exists='replace')

# Perform SQL query to join tables
# Replace 'common_column' with the actual name of the common column
query = """
SELECT *
FROM movies
INNER JOIN credits ON movies.title = credits.title
"""

dfcombined = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

df = dfcombined[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

# See info
print(df.info())

# See data
print(df.head())

# Check for duplicates
print(f'''Duplicated: {df.duplicated().sum()}''')
duplicates = df[df.duplicated()]
print(duplicates)

# Drop duplicates
df.drop_duplicates(inplace=True)

# Function to extract 'name' from JSON objects
def extract_names(json_str):
    try:
        json_objects = json.loads(json_str)
        return [obj['name'] for obj in json_objects]
    except json.JSONDecodeError:
        return []

# Function to extract the first three cast names
def extract_cast_names(json_str):
    try:
        json_objects = json.loads(json_str)
        return [obj['name'] for obj in json_objects[:3]]
    except json.JSONDecodeError:
        return []

# Function to extract director's name
def extract_director_name(json_str):
    try:
        json_objects = json.loads(json_str)
        for obj in json_objects:
            if obj['job'] == 'Director':
                return obj['name']
        return None
    except json.JSONDecodeError:
        return None

# Apply transformations
df['genres'] = df['genres'].apply(extract_names)
df['keywords'] = df['keywords'].apply(extract_names)
df['cast'] = df['cast'].apply(extract_cast_names)
df['crew'] = df['crew'].apply(extract_director_name)

# Replace NaN values in 'overview' with empty strings
df['overview'] = df['overview'].fillna('')
df['overview'] = df['overview'].apply(lambda x: x.split())  # Splitting overview into a list of words




# Combine the columns into one 'tags' column
df['tags'] = df.apply(lambda row: ','.join(str(v) for v in row[['genres', 'keywords', 'cast', 'crew', 'overview']]), axis=1)

# Replace commas with blanks (spaces)
df['tags'] = df['tags'].str.replace(',', ' ')

# Create a new dataframe or modify the existing one
copy_df = df.copy() 

# Checking for duplicates specifically in the 'title' column
print(f"Duplicate titles: {copy_df['title'].duplicated().sum()}")

# Find rows with duplicate values in any column
duplicates_any = pd.concat([copy_df[col].duplicated() for col in copy_df.columns], axis=1).any(axis=1)

# Drop these rows
new_df = copy_df[~duplicates_any]

# Reset the index
new_df.reset_index(drop=True, inplace=True)


# Check the first entry in the 'tags' column
print(new_df["tags"][0])

print("Duplicate titles before removal:", new_df['title'].duplicated().sum())
new_df = new_df.drop_duplicates(subset='title', keep='first')
print("Duplicate titles after removal:", new_df['title'].duplicated().sum())
new_df.reset_index(drop=True, inplace=True)


# Convert text in 'tags' column to vectors
vectorizer = CountVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(new_df['tags']).toarray()

# Calculate similarity
similarity = cosine_similarity(vectors)

def recommend(movie):
    # Check if the movie exists in the DataFrame
    if movie not in new_df['title'].values:
        print(f"Movie '{movie}' not found.")
        return

    # Fetch the index of the movie in a safer way
    try:
        movie_index = new_df.query('title == @movie').index[0]
    except IndexError:
        print(f"Movie '{movie}' index not found.")
        return

    # Retrieve similarity scores
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    # Print recommended movies
    for i in movie_list:
        print(new_df.iloc[i[0]].title)


recommend("Avatar")






<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

/tmp/ipykernel_3658/3632309629.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)
/tmp/ipykernel_3658/3632309629.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres'] = df['genres'].apply(extract_names)
/tmp/ipykernel_3658/3632309629.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

Duplicate titles: 9
['Action'  'Adventure'  'Fantasy'  'Science Fiction'] ['culture clash'  'future'  'space war'  'space colony'  'society'  'space travel'  'futuristic'  'romance'  'space'  'alien'  'tribe'  'alien planet'  'cgi'  'marine'  'soldier'  'battle'  'love affair'  'anti war'  'power relations'  'mind and soul'  '3d'] ['Sam Worthington'  'Zoe Saldana'  'Sigourney Weaver'] James Cameron ['In'  'the'  '22nd'  'century '  'a'  'paraplegic'  'Marine'  'is'  'dispatched'  'to'  'the'  'moon'  'Pandora'  'on'  'a'  'unique'  'mission '  'but'  'becomes'  'torn'  'between'  'following'  'orders'  'and'  'protecting'  'an'  'alien'  'civilization.']
Duplicate titles before removal: 0
Duplicate titles after removal: 0
title    Moonraker
title    Moonraker
Name: 308, dtype: object
title    Lost in Space
title    Lost in Space
Name: 153, dtype: object
title    John Carter
title    John Carter
Name: 4, dtype: object
title    The Hitchhiker's Guide to the Galaxy
title    The Hitchhiker